In [0]:
import pandas
import google
import os
import pickle
import numpy as np
import xml.etree.ElementTree as ET # for parsing XML
import matplotlib.pyplot as plt # to show images
from PIL import Image # to read images
import glob
from numpy import asarray

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import multiprocessing as mp

In [0]:
os.chdir('drive/My Drive/Volvo/Data/JAAD')

In [0]:
#Helper functions
def frame_start_crossing(behavior):
  first_index = -1
  for i, frame in enumerate(behavior):
    if frame == 1:
      first_index = i
      return i, 1
  # Pedestrian does not cross
  return -1, 0


def get_images(video,frames):
  os.chdir('images')
  os.chdir(video)
  list_all_images = os.listdir()
  list_images = list_all_images[min(frames):max(frames)+1]
  os.chdir('..')
  os.chdir('..')
  return list_images

def crop_image(video,image,bbox):
  os.chdir('images')
  os.chdir(video)
  im=Image.open(image)
  im=im.crop(bbox)
  os.chdir('..')
  os.chdir('..') 
  return im

def get_data_split():
  os.chdir('data_split')
  a = open('train.txt')
  b = open('test.txt')
  c = open('val.txt')
  train = a.read().split('\n') + c.read().split('\n')
  test = b.read().split('\n') 
  test.sort()
  os.chdir('..')
  return train, test

def inDataset(video, train, test, wantTrainingSet):
  # 251, 252, 255, 256 didn't work in the original script since the annotations weren't correctly formatted. 
  # But let's include them in the ground truth
  # broken = ["video_0251", "video_0252", "video_0255", "video_0256"]
  # if video in broken:
  #  return False
  if wantTrainingSet:
    return video in train
  else:
    return int(video[-3:]) >= 233 #video in test

def saveData(data, filename, wantTrainingSet):
  os.chdir('ground_truth')
  if wantTrainingSet:
    os.chdir('train')
  else:
    os.chdir('test')

  np.save(filename, data)
  os.chdir('../..')

In [0]:
# Import JAAD dataset
os.chdir('data_cache')
pickle_in = open("jaad_database.pkl","rb")
jaad = pickle.load(pickle_in)
os.chdir('..')

In [7]:
# Get test/train split
train, test = get_data_split()
print(train)
print(test)


['video_0001', 'video_0003', 'video_0004', 'video_0007', 'video_0008', 'video_0009', 'video_0010', 'video_0011', 'video_0012', 'video_0013', 'video_0014', 'video_0018', 'video_0019', 'video_0020', 'video_0024', 'video_0025', 'video_0026', 'video_0027', 'video_0030', 'video_0031', 'video_0033', 'video_0034', 'video_0035', 'video_0037', 'video_0038', 'video_0039', 'video_0047', 'video_0049', 'video_0050', 'video_0051', 'video_0052', 'video_0054', 'video_0056', 'video_0057', 'video_0060', 'video_0061', 'video_0062', 'video_0064', 'video_0066', 'video_0069', 'video_0074', 'video_0077', 'video_0078', 'video_0079', 'video_0080', 'video_0081', 'video_0083', 'video_0085', 'video_0086', 'video_0088', 'video_0091', 'video_0094', 'video_0095', 'video_0098', 'video_0108', 'video_0109', 'video_0111', 'video_0112', 'video_0114', 'video_0119', 'video_0120', 'video_0121', 'video_0122', 'video_0126', 'video_0129', 'video_0130', 'video_0131', 'video_0132', 'video_0133', 'video_0134', 'video_0136', 'vide

In [0]:
# Connect videos to bounding box

def getDataSet(jaad, wantTrainingSet):
  # Change to True if want train set, False for test set
  # 1-252 is train, 253-345 is test (~70/30 split)
  # Note: Each frame contains many pedestrians. Test contains 217, train contains

  list_videos = jaad.keys()
  print("Total number of videos in JAAD: ", len(list_videos))

  # The script will take a while to run, use this to test a few for debug
  # list_videos = ['video_0005', 'video_0016', 'video_0022']
  # list_videos = ["video_0251", "video_0252", "video_0255", "video_0256"]

  # m x 6 matrix: Frame num, Bbox xmin, Bbox ymin, Bbox xmax, Bboxymax, Crossing=1
  # Each row represents the first frame that a pedestrian crosses
  
  for video in list_videos:
    # Generate the test/train set accordingly
    if not inDataset(video, train, test, wantTrainingSet):
      continue

    crossingPedestrians = []
    pedestrian_list = jaad[video]['ped_annotations'].keys()
    print(pedestrian_list)
    for pedestrian_id in pedestrian_list:
    # It's only helpful if the pedestrian has behavioral tags
      if pedestrian_id[-1] != 'b': 
        continue

      cross_behavior = jaad[video]['ped_annotations'][pedestrian_id]['behavior']['cross']
      frames = jaad[video]['ped_annotations'][pedestrian_id]['frames']
      bounding_boxes = jaad[video]['ped_annotations'][pedestrian_id]['bbox']

      cross_index, crossing_label = frame_start_crossing(cross_behavior)
      # When generating ground truth, disregard if pedestrian is not crossing
      if crossing_label == 0:
        continue
      bbox = bounding_boxes[cross_index]
      crossingPedestrians.append([cross_index, bbox[0], bbox[1], bbox[2], bbox[3], 1])
    crossingPedestrians = sorted(crossingPedestrians, key=lambda x: x[0])
    if len(crossingPedestrians) > 0:
      saveData(crossingPedestrians, video, wantTrainingSet)
    print("Finished video [%s / %d] - Pedestrians crossed: %d" % (video[-3:],  (220 if wantTrainingSet else 126), len(crossingPedestrians)))
  return #np.array(crossingPedestrians)

In [9]:
wantTrainingSet = False
test_ground_truth = getDataSet(jaad, wantTrainingSet)

Total number of videos in JAAD:  345
dict_keys(['0_233_1791b', '0_233_1778', '0_233_1779', '0_233_1780', '0_233_1781', '0_233_1782', '0_233_1783', '0_233_1784', '0_233_1785', '0_233_1786b', '0_233_1787', '0_233_1788', '0_233_1789', '0_233_1790', '0_233_1791'])
Finished video [233 / 126] - Pedestrians crossed: 2
dict_keys(['0_234_1804b', '0_234_1797b', '0_234_1794', '0_234_1795', '0_234_1796', '0_234_1797', '0_234_1798', '0_234_1799', '0_234_1800b', '0_234_1801', '0_234_1802', '0_234_1803', '0_234_1804'])
Finished video [234 / 126] - Pedestrians crossed: 3
dict_keys(['0_235_1813b', '0_235_1810b', '0_235_1807', '0_235_1807b', '0_235_1809', '0_235_1810', '0_235_1811', '0_235_1812', '0_235_1813', '0_235_1814'])
Finished video [235 / 126] - Pedestrians crossed: 2
dict_keys(['0_236_1829b', '0_236_1820b', '0_236_1817', '0_236_1818', '0_236_1819', '0_236_1820', '0_236_1821', '0_236_1822', '0_236_1823', '0_236_1824b', '0_236_1825', '0_236_1826', '0_236_1827', '0_236_1828', '0_236_1829'])
Finish

In [0]:
wantTrainingSet = True
train_ground_truth = getDataSet(jaad, wantTrainingSet)

Total number of videos in JAAD:  345
dict_keys(['0_252_1968', '0_252_1969', '0_252_1970', '0_252_1971', '0_252_1968b'])
Finished video [252 / 220] - Pedestrians crossed: 0
dict_keys(['0_255_1985b', '0_255_1980', '0_255_1981', '0_255_1982', '0_255_1983', '0_255_1984', '0_255_1985', '0_255_1986'])
Finished video [255 / 220] - Pedestrians crossed: 1
dict_keys(['0_256_1987', '0_256_1987b'])
Finished video [256 / 220] - Pedestrians crossed: 1


In [0]:
# os.chdir('data_cache')
os.chdir('..')
os.getcwd()

'/content/drive/My Drive/Volvo/Data/JAAD'

In [0]:
os.chdir('..')

In [0]:
# Load in data
os.chdir('ground_truth/test')
truth = np.load('video_0016.npy')
print(truth)
os.chdir('..')
os.chdir('..')


[[  0. 759. 655. 779. 711.   1.]
 [  0. 823. 654. 845. 709.   1.]]


In [0]:
mp.get_start_method

<module 'multiprocessing.connection' from '/usr/lib/python3.6/multiprocessing/connection.py'>